### Könyvtárak importálása

In [1]:
import pandas as pd
import numpy as np
import researchpy as rp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
ldir = os.chdir(r'C:\Users\Gergely\Documents\Datasets\Forma1')

### Drivers dataset

In [2]:
df_drivers = pd.read_csv('drivers.csv', parse_dates=['dob']).drop(columns=['driverRef', 'number', 'code', 'url'])
df_drivers.head()

,driverId,forename,surname,dob,nationality
0,1,Lewis,Hamilton,1985-01-07,British
1,2,Nick,Heidfeld,1977-05-10,German
2,3,Nico,Rosberg,1985-06-27,German
3,4,Fernando,Alonso,1981-07-29,Spanish
4,5,Heikki,Kovalainen,1981-10-19,Finnish


In [3]:
df_drivers.shape

(847, 5)

In [4]:
df_drivers.info(null_counts=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 5 columns):
driverId       int64
forename       object
surname        object
dob            datetime64[ns]
nationality    object
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 33.2+ KB


### Results dataset

In [5]:
df_results = pd.read_csv('results.csv').drop(columns=['number', 'time', 'milliseconds', 'fastestLap', 'rank'])
df_results.head()

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,laps,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,1,1,1,1,10.0,58,1:27.452,218.300,1
1,2,18,2,2,5,2,2,2,8.0,58,1:27.739,217.586,1
2,3,18,3,3,7,3,3,3,6.0,58,1:28.090,216.719,1
3,4,18,4,4,11,4,4,4,5.0,58,1:28.603,215.464,1
4,5,18,5,1,3,5,5,5,4.0,58,1:27.418,218.385,1


In [6]:
df_results.shape

(24620, 13)

### df = Drivers & Results

In [7]:
df = pd.merge(df_drivers, df_results, on='driverId', how='left')
df.head()

,driverId,forename,surname,dob,nationality,resultId,raceId,constructorId,grid,position,positionText,positionOrder,points,laps,fastestLapTime,fastestLapSpeed,statusId
0,1,Lewis,Hamilton,1985-01-07,British,1,18,1,1,1,1,1,10.0,58,1:27.452,218.300,1
1,1,Lewis,Hamilton,1985-01-07,British,27,19,1,9,5,5,5,4.0,56,1:35.462,209.033,1
2,1,Lewis,Hamilton,1985-01-07,British,57,20,1,3,13,13,13,0.0,56,1:35.520,203.969,11
3,1,Lewis,Hamilton,1985-01-07,British,69,21,1,5,3,3,3,6.0,66,1:22.017,204.323,1
4,1,Lewis,Hamilton,1985-01-07,British,90,22,1,3,2,2,2,8.0,58,1:26.529,222.085,1


In [8]:
pontok_szum = dict(df.groupby(by='driverId').points.sum())
pontok_szum = {k: v for k, v in sorted(pontok_szum.items(), key=lambda item: item[1], reverse=True)}
pontok_szum

{1: 3431.0,
 20: 2985.0,
 4: 1899.0,
 8: 1859.0,
 3: 1594.5,
 30: 1566.0,
 822: 1289.0,
 18: 1235.0,
 13: 1167.0,
 17: 1047.5,
 817: 1040.0,
 830: 948.0,
 117: 798.5,
 22: 658.0,
 102: 614.0,
 815: 581.0,
 14: 535.0,
 807: 511.0,
 137: 485.5,
 95: 482.0,
 182: 420.5,
 57: 420.0,
 154: 389.0,
 77: 385.0,
 71: 360.0,
 328: 360.0,
 23: 329.0,
 199: 310.0,
 31: 307.0,
 844: 303.0,
 289: 289.0,
 119: 281.0,
 224: 281.0,
 579: 279.14,
 21: 275.0,
 9: 274.0,
 373: 274.0,
 832: 267.0,
 356: 261.0,
 2: 259.0,
 222: 255.0,
 304: 248.0,
 15: 246.5,
 55: 241.0,
 35: 235.0,
 172: 228.0,
 223: 212.0,
 178: 206.0,
 238: 206.0,
 360: 196.5,
 56: 191.0,
 105: 186.5,
 475: 186.5,
 163: 181.0,
 235: 181.0,
 207: 180.0,
 341: 180.0,
 231: 179.0,
 49: 174.0,
 826: 170.0,
 187: 169.0,
 177: 159.5,
 825: 157.0,
 221: 141.0,
 647: 140.0,
 839: 136.0,
 364: 133.0,
 123: 132.0,
 578: 127.64,
 642: 127.33,
 155: 125.0,
 16: 124.0,
 842: 124.0,
 173: 122.0,
 814: 121.0,
 358: 109.0,
 203: 107.0,
 386: 107.0,
 5: 

In [9]:
for i, x in enumerate(df['position']):
    if x == '\\N':
        df.at[i, 'position'] = 100
    else:
        df.at[i, 'position'] = x

In [10]:
for i, x in enumerate(df['position']):
    df.at[i, 'position'] = int(df.at[i, 'position'])

In [11]:
df['position'] = df['position'].astype('Int16')

In [12]:
for i, x in enumerate(df['position']):
    if x < 11:
        df.at[i, 'pointsTen'] = 11-df.at[i, 'position']
    else:
        df.at[i, 'pointsTen'] = 0

In [13]:
df.head()

,driverId,forename,surname,dob,nationality,resultId,raceId,constructorId,grid,position,positionText,positionOrder,points,laps,fastestLapTime,fastestLapSpeed,statusId,pointsTen
0,1,Lewis,Hamilton,1985-01-07,British,1,18,1,1,1,1,1,10.0,58,1:27.452,218.300,1,10.0
1,1,Lewis,Hamilton,1985-01-07,British,27,19,1,9,5,5,5,4.0,56,1:35.462,209.033,1,6.0
2,1,Lewis,Hamilton,1985-01-07,British,57,20,1,3,13,13,13,0.0,56,1:35.520,203.969,11,0.0
3,1,Lewis,Hamilton,1985-01-07,British,69,21,1,5,3,3,3,6.0,66,1:22.017,204.323,1,8.0
4,1,Lewis,Hamilton,1985-01-07,British,90,22,1,3,2,2,2,8.0,58,1:26.529,222.085,1,9.0


### 10-es rendszerben a legtöbb pontot szerző versenyző: Michael Scumacher

In [14]:
pontok10_szum = dict(df.groupby(by='driverId').pointsTen.sum())
pontok10_szum = {k: v for k, v in sorted(pontok10_szum.items(), key=lambda item: item[1], reverse=True)}
pontok10_szum

{30: 1796.0,
 1: 1757.0,
 20: 1544.0,
 8: 1475.0,
 4: 1411.0,
 117: 1156.0,
 22: 1029.0,
 18: 1020.0,
 13: 960.0,
 14: 922.0,
 3: 899.0,
 102: 869.0,
 137: 845.0,
 77: 755.0,
 17: 753.0,
 95: 713.0,
 57: 694.0,
 822: 683.0,
 182: 639.0,
 23: 617.0,
 119: 614.0,
 55: 591.0,
 199: 585.0,
 817: 568.0,
 21: 556.0,
 289: 542.0,
 71: 520.0,
 328: 507.0,
 224: 506.0,
 830: 502.0,
 172: 501.0,
 304: 497.0,
 15: 496.0,
 356: 473.0,
 35: 461.0,
 222: 461.0,
 49: 451.0,
 2: 447.0,
 223: 444.0,
 56: 440.0,
 31: 427.0,
 105: 423.0,
 187: 412.0,
 360: 404.0,
 579: 385.0,
 123: 382.0,
 163: 377.0,
 815: 376.0,
 238: 375.0,
 373: 373.0,
 178: 354.0,
 235: 354.0,
 341: 343.0,
 207: 342.0,
 807: 338.0,
 84: 328.0,
 177: 320.0,
 173: 319.0,
 65: 317.0,
 231: 314.0,
 221: 303.0,
 9: 284.0,
 475: 281.0,
 175: 277.0,
 44: 269.0,
 364: 258.0,
 118: 249.0,
 200: 243.0,
 154: 242.0,
 578: 241.0,
 386: 232.0,
 278: 229.0,
 202: 228.0,
 140: 221.0,
 306: 221.0,
 642: 221.0,
 427: 220.0,
 110: 219.0,
 158: 216.0,

In [45]:
df_drivers[df_drivers['driverId'] == 30]

,driverId,forename,surname,dob,nationality
29,30,Michael,Schumacher,1969-01-03,German


In [18]:
df_qualifying = pd.read_csv('qualifying.csv').drop(columns=['qualifyId', 'number'])
df_qualifying.head()

,raceId,driverId,constructorId,position,q1,q2,q3
0,18,1,1,1,1:26.572,1:25.187,1:26.714
1,18,9,2,2,1:26.103,1:25.315,1:26.869
2,18,5,1,3,1:25.664,1:25.452,1:27.079
3,18,13,6,4,1:25.994,1:25.691,1:27.178
4,18,2,2,5,1:25.960,1:25.518,1:27.236


In [19]:
df = pd.merge(df, df_qualifying, left_on=['raceId','driverId', 'constructorId'], right_on = ['raceId','driverId', 'constructorId'], how='left')

In [20]:
df.head()

,driverId,forename,surname,dob,nationality,resultId,raceId,constructorId,grid,position_x,...,points,laps,fastestLapTime,fastestLapSpeed,statusId,pointsTen,position_y,q1,q2,q3
0,1,Lewis,Hamilton,1985-01-07,British,1,18,1,1,1,...,10.0,58,1:27.452,218.300,1,10.0,1.0,1:26.572,1:25.187,1:26.714
1,1,Lewis,Hamilton,1985-01-07,British,27,19,1,9,5,...,4.0,56,1:35.462,209.033,1,6.0,4.0,1:35.392,1:34.627,1:36.709
2,1,Lewis,Hamilton,1985-01-07,British,57,20,1,3,13,...,0.0,56,1:35.520,203.969,11,0.0,3.0,1:32.750,1:31.922,1:33.292
3,1,Lewis,Hamilton,1985-01-07,British,69,21,1,5,3,...,6.0,66,1:22.017,204.323,1,8.0,5.0,1:21.366,1:20.825,1:22.096
4,1,Lewis,Hamilton,1985-01-07,British,90,22,1,3,2,...,8.0,58,1:26.529,222.085,1,9.0,3.0,1:26.192,1:26.477,1:27.923


In [22]:
df[df['nationality'] == 'Hungarian']

,driverId,forename,surname,dob,nationality,resultId,raceId,constructorId,grid,position_x,...,points,laps,fastestLapTime,fastestLapSpeed,statusId,pointsTen,position_y,q1,q2,q3
5551,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1533,90,18,17,100,...,0.0,13,1:30.621,210.666,10,0.0,17.0,1:30.681,\N,\N
5552,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1550,91,18,17,16,...,0.0,52,1:40.123,199.302,14,0.0,17.0,1:39.272,\N,\N
5553,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1573,92,18,20,100,...,0.0,44,1:34.555,206.241,5,0.0,19.0,1:35.787,\N,\N
5554,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1589,93,18,19,15,...,0.0,58,1:26.075,206.317,14,0.0,18.0,1:46.299,\N,\N
5555,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1614,94,18,20,100,...,0.0,17,1:23.390,199.750,20,0.0,20.0,1:21.470,\N,\N
5556,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1623,95,18,19,9,...,0.0,71,1:21.886,146.838,16,2.0,19.0,1:20.060,\N,\N
5557,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1649,96,18,17,15,...,0.0,57,1:34.666,195.770,13,0.0,17.0,1:34.398,\N,\N
5558,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1664,97,18,18,10,...,0.0,66,1:17.516,202.533,14,1.0,18.0,1:17.064,\N,\N
5559,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1682,98,18,19,8,...,1.0,70,1:14.097,203.668,13,3.0,19.0,1:14.812,\N,\N
5560,47,Zsolt,Baumgartner,1981-01-01,Hungarian,1713,99,18,20,100,...,0.0,31,1:19.659,199.344,20,0.0,20.0,1:18.247,\N,\N


### A legtöbb hétvégén szereplő versenyző: Rubens Barichello

In [43]:
versenyek = dict(df.groupby(by='driverId').driverId.count())
versenyek = {k: v for k, v in sorted(versenyek.items(), key=lambda item: item[1], reverse=True)}
versenyek

{22: 326,
 8: 315,
 4: 314,
 18: 309,
 30: 308,
 13: 271,
 119: 257,
 15: 256,
 1: 250,
 14: 247,
 20: 241,
 21: 231,
 17: 217,
 105: 215,
 110: 214,
 77: 210,
 137: 207,
 3: 206,
 55: 202,
 117: 202,
 95: 192,
 2: 184,
 23: 180,
 172: 180,
 289: 179,
 807: 179,
 815: 178,
 182: 174,
 817: 171,
 154: 166,
 35: 165,
 57: 165,
 65: 165,
 84: 165,
 123: 164,
 163: 164,
 102: 162,
 118: 162,
 49: 159,
 44: 158,
 187: 154,
 224: 149,
 56: 147,
 199: 146,
 158: 143,
 197: 143,
 822: 140,
 223: 138,
 207: 129,
 356: 129,
 16: 128,
 177: 128,
 94: 123,
 238: 123,
 71: 122,
 175: 122,
 235: 121,
 178: 117,
 112: 115,
 200: 114,
 222: 113,
 5: 112,
 304: 112,
 341: 112,
 63: 111,
 160: 111,
 173: 110,
 347: 109,
 37: 107,
 50: 107,
 278: 106,
 140: 103,
 360: 103,
 825: 103,
 830: 102,
 832: 102,
 328: 100,
 346: 100,
 64: 99,
 122: 98,
 9: 97,
 79: 97,
 828: 97,
 813: 96,
 10: 95,
 31: 95,
 221: 95,
 826: 95,
 231: 93,
 11: 91,
 88: 88,
 156: 88,
 176: 88,
 306: 87,
 364: 87,
 427: 87,
 92: 84,

In [52]:
df_drivers[df_drivers['driverId'] == 22]

,driverId,forename,surname,dob,nationality
21,22,Rubens,Barrichello,1972-05-23,Brazilian


### Az összes induló 40%-ának sikerült legalább egyszer a pontszerzés

In [51]:
(df[df['points'] > 0].groupby(by='driverId').count().count() / df.groupby(by='driverId').driverId.count().count()).raceId

0.4037780401416765